<a href="https://colab.research.google.com/github/Bendrox/DA_etude_bonvivre/blob/main/Explore_L%C3%A9giFrance_API_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - API connect

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


In [ ]:
pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session
import json

In [ ]:
#option 2:
import requests

# idenifiants  API
client_id = '2bd5c3ac-5128-4527-b74f-5d3e90e47efb'
client_secret = 'de5cf33d-fbee-4368-8fe9-e803490fe096'

# URL pour obtenir le jeton
token_url = 'https://sandbox-oauth.piste.gouv.fr/api/oauth/token'

# inject cred for token request
token_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'openid'
}

# run the request
response = requests.post(token_url, data=token_data)
response.raise_for_status()  # vérif  erreurs

# récup  jeton
token_info = response.json()
access_token = token_info['access_token']


In [ ]:
print(access_token)

yIGkWxQIXiWEIOI2qK2qgvcCKXXzOzKZo7PLAmvKAY7R8cYJB38vo7


# 2 - API Call

Environ 70 requetes, en 6 "types"/"familles" de requêtes:

1 - Consult Controller

2 - List Controller

3 - Suggest Controller

4 - Misc Controller

5 - Search Controller

6 - Chrono Controller : pour accéder à l'historique et à l'évolution chronologique des textes.

Q : permet de retracer et comprendre la version d'un texte en vigueur à une date donnée ou de suivre ses modifications ?


## 2.0 - Ping/Pong Test

In [ ]:
# Call 1 : Ping/Pong
headers_1=  {
		'accept': 'text/plain',
		'Authorization': 'Bearer ' + access_token
	}

output = requests.get("https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/ping",
                       headers = headers_1).text

display(output)

'pong'

## 2.1 - Consult Controller

### 2.1.1 - Contenu Article - Eli


Eli - Identifiant Européen de la Législation:

- Permet d'identifier de façon unique les documents législatifs et réglementaires au sein de la base de données de Légifrance.

- Déclinaison :

/eli : Préfixe indiquant qu'il s'agit d'un identifiant ELI.

/decret : Type de texte (ici un décret) peut aussi être un arrêté, une loi, ou d'autres types d'actes normatifs...

/2021/7/13 : Date de publication ou adoption (13 juillet 2021 dans cet exemple).

/PRMD2117108D : Numéro unique du décret

/jo : Indication que le document a été publié dans le Journal Officiel.

/article_1 : Référence à un article spécifique dans le document (ici l'article 1).

- Sources:

https://eur-lex.europa.eu/content/help/eurlex-content/eli.html?locale=fr#:~:text=L%27identifiant%20europ%C3%A9en%20de%20la,r%C3%A9utiliser%20par%2Ddel%C3%A0%20les%20fronti%C3%A8res.


https://eur-lex.europa.eu/eli-register/about.html?locale=fr




In [ ]:
# Call 2: Contenu article
headers_2 = {
    "accept": "application/json",
    "Content-Type": "application/json",
    'Authorization': 'Bearer ' + access_token}

data = {
    "idEliOrAlias": "/eli/decret/2021/7/13/PRMD2117108D/jo/article_1"
}

url= "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleWithIdEliOrAlias"
response = requests.post(url,
                         headers=headers_2,
                         json=data) # dictionnaire data converti en JSON
                                    #inject request


In [ ]:
response_json = response.json()
display(response_json) #datetype

# 'etat': 'VIGUEUR'
# JORF
#structure (ex: livre , chapitre, section ...)
# n°article
#titre': "Décret n° 2021-922 du 13 juillet 2021 portant création, auprès du secrétaire général de la défense et de la sécurité nationale, d'un service à compétence nationale dénommé « service de vigilance et de protection contre les ingérences numériques étrangères »",

{'executionTime': 0,
 'dereferenced': False,
 'article': {'id': 'JORFARTI000043788371',
  'idTexte': None,
  'type': 'ENTIEREMENT_MODIF',
  'texte': "Le code de la défense est ainsi modifié : 1° L'article R.* 1132-3 est complété par un 9° ainsi rédigé : « 9° En liaison avec les départements ministériels concernés, il identifie les opérations impliquant, de manière directe ou indirecte, un Etat étranger ou une entité non étatique étrangère, et visant à la diffusion artificielle ou automatisée, massive et délibérée, par le biais d'un service de communication au public en ligne, d'allégations ou imputations de faits manifestement inexactes ou trompeuses de nature à porter atteinte aux intérêts fondamentaux de la Nation. Il anime et coordonne les travaux interministériels en matière de protection contre ces opérations. » ; 2° La section 1 du chapitre II du titre III du livre premier de la première partie est complétée par un article D. 1132-8 ainsi rédigé : « Art. D. 1132-8. - Pour assurer

In [ ]:
response_json['article']['textTitles'][1]["titreLong"]

"Décret n° 2021-922 du 13 juillet 2021 portant création, auprès du secrétaire général de la défense et de la sécurité nationale, d'un service à compétence nationale dénommé « service de vigilance et de protection contre les ingérences numériques étrangères »"

In [ ]:
response_json['article']['idEliAlias']

'/eli/decret/2021/7/13/2021-922/jo/article_1'

In [ ]:
response_json['article']['texte']

"Le code de la défense est ainsi modifié : 1° L'article R.* 1132-3 est complété par un 9° ainsi rédigé : « 9° En liaison avec les départements ministériels concernés, il identifie les opérations impliquant, de manière directe ou indirecte, un Etat étranger ou une entité non étatique étrangère, et visant à la diffusion artificielle ou automatisée, massive et délibérée, par le biais d'un service de communication au public en ligne, d'allégations ou imputations de faits manifestement inexactes ou trompeuses de nature à porter atteinte aux intérêts fondamentaux de la Nation. Il anime et coordonne les travaux interministériels en matière de protection contre ces opérations. » ; 2° La section 1 du chapitre II du titre III du livre premier de la première partie est complétée par un article D. 1132-8 ainsi rédigé : « Art. D. 1132-8. - Pour assurer les missions qui lui sont assignées au 9° de l'article R.* 1132-3, le secrétaire général de la défense et de la sécurité nationale dispose du servic

## 2.6 - Chrono Controller

1 - Extrait d'une version d'un texte:
POST
/chrono/textCidAndElementCid - ok

2 - Vérifie si un texte possède des versions:
GET
/chrono/textCid/{textCid} - ok

3 - Version d'un texte:
POST
/chrono/textCid - ok

4 - Test controller:
GET
/chrono/ping


### 2.6.1  Extrait d'une version d'un texte : chrono/textCidAndElementCid

Récupère un extrait (section ou article) d'une version spécifique d'un texte à partir des identifiants du texte et de l'extrait (section ou article)


In [ ]:
import requests

headers = {
		'accept': 'application/json',
		'Content-Type': 'application/json',
		'Authorization': 'Bearer ' + access_token
	}

url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCidAndElementCid"

# elementCid  est un identifiant unique  pour référencer un contenu juridique spécifique,
# exemple  article de loi,  section,  chapitre ou  autre subdivision juridique.

data = {
    "textCid": "LEGITEXT000006072050", # code du travail
    "elementCid": "LEGIARTI000006901817"
}
# Test 1: "textCid": "LEGITEXT000006070721" Code Civil, "elementCid": "LEGIARTI000038033084" -> NA
# Test 2: "textCid": "LEGITEXT000006070721" Code Civil , "elementCid": "LEGIARTI000006417324" -> NA
# Test 3 : "textCid": "LEGITEXT000006072050" Code du Travail + Article L1221-1 "elementCid": "LEGIARTI000006901817" -> ok
# https://www.legifrance.gouv.fr/codes/article_lc/LEGIARTI000006900839

In [ ]:
#  requête GET
response = requests.post(url, headers=headers, json= data)
data = response.json()
display(data)

{'executionTime': 0,
 'regroupements': [{'title': '2008',
   'versions': {'2008-05-01': {'dateDebut': 1209600000000,
     'isEndVersion': False,
     'articlesModificateurs': {'JORFTEXT000000465978': {'title': 'Ordonnance 2007-329 2007-03-12 JORF 13 mars 2007',
       'id': 'JORFTEXT000000465978',
       'nature': 'CODE',
       'dateDebutCible': 32472144000000,
       'actions': {'CODIFICATION': {'action': 'CODIFICATION',
         'parents': {'LEGISCTA000006189538': {'name': 'Section 2 : Information et communication.',
           'id': 'LEGISCTA000006189538',
           'cid': 'LEGISCTA000006189538',
           'cidText': 'LEGITEXT000006072050',
           'nature': 'CODE',
           'dateDebut': 1209600000000,
           'articlesCibles': {'LEGIARTI000006901817': {'name': 'L2262-8',
             'id': 'LEGIARTI000006901817',
             'cid': 'LEGIARTI000006901817',
             'idParent': 'LEGISCTA000006189538',
             'cidParent': 'LEGISCTA000006189538',
             'cid

### 2.6.2  Version d'un texte /chrono/textCid (Binary output)


In [ ]:
import requests

# def text_cid
# LEGITEXT000006073984 # code des assurances : oui
# LEGITEXT000006070721 # code civil : oui
# LEGITEXT000005634379 # Code de commerce : oui
# LEGITEXT000006070719 # code pénal  oui
# LEGITEXT000006072026 # cmf

text_cid = "LEGITEXT000006073984" # input

# URL de l'API + inject textCid
url = f"https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCid/{text_cid}"


headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + access_token
}

# Envoi  requête GET
response = requests.get(url, headers=headers)
data = response.json()
display(data)


{'executionTime': 2, 'chronolegiExists': True}

In [ ]:
type(data)

dict

In [ ]:
def version_exist(text_cid):
  url = f"https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCid/{text_cid}"
  headers = {
    "accept": "application/json",
        "Authorization": "Bearer " + access_token}
  response = requests.get(url, headers=headers)
  data = response.json()
  return display(data["chronolegiExists"])

In [ ]:
version_exist("LEGITEXT000006073984")
# LEGITEXT000006073984 # code des assurances : oui
# LEGITEXT000006070721 # code civil : oui
# LEGITEXT000005634379 # Code de commerce : oui
# LEGITEXT000006070719 # code pénal  oui
# LEGITEXT000006072026 # cmf

True

### 2.6.3 - Récupère une version d'un texte à partir de l'id texte /chrono/textCid




In [ ]:
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + access_token
}

# Données de la requête
data = {
  "endYear": 2018,
  "dateConsult": "2021-04-15",
  "startYear": 2015,
  "textCid": "LEGITEXT000006070721"
}

# Envoi de la requête POST
response = requests.post(url, headers=headers, json=data)
display(response)

<Response [200]>

In [ ]:
display(response.json())

{'executionTime': 52,
 'regroupements': [{'title': '2026',
   'versions': {'2026-12-31': {'dateDebut': 1798675200000,
     'isEndVersion': False,
     'articlesModificateurs': {}},
    '2026-01-01': {'dateDebut': 1767225600000,
     'isEndVersion': False,
     'articlesModificateurs': {}}},
   'detailLoaded': False},
  {'title': '2024',
   'versions': {'2024-09-14': {'dateDebut': 1726272000000,
     'isEndVersion': False,
     'articlesModificateurs': {}},
    '2024-07-05': {'dateDebut': 1720137600000,
     'isEndVersion': False,
     'articlesModificateurs': {}},
    '2024-06-15': {'dateDebut': 1718409600000,
     'isEndVersion': False,
     'articlesModificateurs': {}},
    '2024-06-02': {'dateDebut': 1717286400000,
     'isEndVersion': False,
     'articlesModificateurs': {}},
    '2024-04-17': {'dateDebut': 1713312000000,
     'isEndVersion': False,
     'articlesModificateurs': {}},
    '2024-03-20': {'dateDebut': 1710892800000,
     'isEndVersion': False,
     'articlesModificate